# ***Directory Structure***
```
your_project_folder/
├── app.py                      # Your main Flask application file
├── chatbot.py                  # Your chatbot logic file
├── requirements.txt            # Python dependencies for both the website and chatbot
│
├── fine_tuned_chatbot_model/   # Folder for chatbot's model files
│   ├── vocab.txt               # Tokenizer vocabulary
│   ├── special_tokens_map.json
│   ├── tokenizer_config.json
│   └── label_encoder.json      # Encoded labels for the chatbot model
│
├── mnist_cnn_model.h5          # Your handwritten digit recognition model
│
│
├── quantized_chatbot_model.onnx # Your ONNX-format chatbot model
├── new_json.json               # JSON file containing chatbot responses
│
├── static/
│   ├── css/
│   │   └── style.css           # Your updated CSS file with chatbot pop-up styles
│   │
│   ├── js/
│   │   └── script.js           # Your updated JS file with the chatbot toggle logic
│   │
│   └── images/
│       ├── bot.gif             # Your chatbot icon GIF
│       └── favicon.ico         # Your website favicon
│
└── templates/
    ├── index.html              # Your updated HTML file with the chatbot pop-up structure
    └── result.html             # The HTML file for showing digit detection results
```

In [ ]:
!npm install localtunnel
!wget https://raw.githubusercontent.com/codingwithASM/Handwritten_digit_recognition/refs/heads/main/required_files.zip
!unzip required_files.zip
import os
import io
import subprocess
import time
import re
import shutil
from base64 import b64decode

def create_flask_app_files_v3():

    # --- 1. Create Directories ---
    directories = [
        "static/css",
        "static/js",
        "static/images", # For icons, background elements, etc.
        "templates",
        "uploads"
    ]
    for d in directories:
        os.makedirs(d, exist_ok=True)
    shutil.copy('bot.gif', 'static/images/bot.gif')
    shutil.copy('favicon.ico', 'static/images/favicon.ico')
    !unzip fine_tuned_chatbot_model.zip

    # --- 2. Create app.py ---
    app_py_content = """
from flask import Flask, render_template, request, send_file, redirect, url_for, flash, send_from_directory,jsonify
import os
from werkzeug.utils import secure_filename
from PIL import Image
import io
import cv2  # for reading and processing the image(OpenCV)
import numpy as np  # for math operations
import tensorflow as tf # to load trained model
import matplotlib.pyplot as plt #to display the result image

# Import the chatbot functions
from chatbot import chatbot_response # Assuming chatbot.py is in the same directory


app = Flask(__name__)
app.secret_key = os.urandom(24).hex()
app.config['UPLOAD_FOLDER'] = 'uploads/'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# --- Load your ML Model ---
# IMPORTANT: Upload your trained model file (e.g., 'your_ml_model.h5')
MODEL_PATH = 'mnist_cnn_model.h5'

ml_model = None
if os.path.exists(MODEL_PATH):
    ml_model = tf.keras.models.load_model(MODEL_PATH) #load the trained model


# --- Helper function to preprocess image for ML model ---
def preprocess_image(image_file_path):
  if os.path.exists(image_file_path):
    image=cv2.imread(image_file_path,cv2.IMREAD_GRAYSCALE) #load image in grayscale
    blur=cv2.GaussianBlur(image,(5,5),0)#Remove noise
    _,thresh=cv2.threshold(blur,128,255,cv2.THRESH_BINARY_INV) #Convert to black/white (digits=white)
    plt.imshow(thresh)
    return image,thresh
  else:
    flash("File does not exists")
    return None,None


def extract_and_predict_digits(original_image, thresh_image):
  contours,_=cv2.findContours(thresh_image,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) #contours are the outlines around each digit
  annotated_image=cv2.cvtColor(original_image.copy(),cv2.COLOR_GRAY2BGR)#Go through each digit found, crop it and predict it.

  for cnt in contours:
    x,y,w,h=cv2.boundingRect(cnt)
    if cv2.contourArea(cnt)>=30 and w>=1 and h>=3: #find box(x,y,w,h) around digit and ignore very small contours (likely noise)
      roi=thresh_image[y:y+h,x:x+w] #crop digit(ROI= region of interest)
      roi_resized=cv2.resize(roi,(20,20),interpolation=cv2.INTER_AREA)  #resize to 28x28 (like MNIST)

      roi_resized=np.pad(roi_resized,((4,4),(4,4)),'constant',constant_values=0)
      roi_resized=roi_resized.astype("float32")/255.0 #normalize
      roi_reshaped=roi_resized.reshape(1,28,28,1)#reshape for model input

      prediction=ml_model.predict(roi_reshaped)  #predict the digit
      label=np.argmax(prediction) #gives the index (0-9) with the highest probability
      cv2.rectangle(annotated_image,(x,y),(x+w,y+h),(0,255,0),2)
      cv2.putText(annotated_image,str(label),(x,y+2),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,255,0),2) #Draw a green rectangle around the digit and put the predicted number above it.
  return(annotated_image)

# --- Flask Routes ---

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        flash('No file part in the request.', 'error')
        return redirect(request.url)

    file = request.files['file']
    if file.filename == '':
        flash('No image selected.', 'error')
        return redirect(request.url)

    if not allowed_file(file.filename):
        flash('Invalid file type. Only PNG, JPG, JPEG images are allowed.', 'error')
        return redirect(request.url)

    if ml_model is None:
        flash("ML model is not loaded. Please upload 'your_ml_model.h5' ", 'error')
        return redirect(url_for('index'))

    filename = secure_filename(file.filename)
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(filepath)

    try:
        original,thresholded = preprocess_image(filepath)
        if original is None or thresholded is None:
            flash("Failed to preprocess image. Please try another image.", 'error')
            return redirect(url_for('index'))


        processed_image_name = "predicted_" + filename
        processed_image_path = os.path.join(app.config['UPLOAD_FOLDER'], processed_image_name)
        result_img=extract_and_predict_digits(original, thresholded)
        result_img=cv2.cvtColor(result_img,cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(8,6))
        plt.imshow(result_img)
        plt.title("Detected digits with predictions")
        plt.axis('off')
        plt.savefig(processed_image_path, bbox_inches='tight', pad_inches=0)
        plt.close() # Close the plot to free memory
        return render_template('result.html', prediction_file=processed_image_name)

    except Exception as e:
        flash(f"An error occurred during prediction: {e}", 'error')
        return redirect(url_for('index'))
    finally:
        if os.path.exists(filepath):        # Clean up the temporarily uploaded image
            os.remove(filepath)

@app.route('/download/<filename>')
def download_file(filename):
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    if os.path.exists(file_path):
        return send_file(file_path, as_attachment=True, download_name=filename)
    else:
        flash("Download file not found.", 'error')
        return redirect(url_for('index'))

def allowed_file(filename):
    return '.' in filename and \\
           filename.rsplit('.', 1)[1].lower() in {'png', 'jpg', 'jpeg'}

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)


# --- Chatbot Route ---
@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    if user_message:
        response = chatbot_response(user_message)
        return jsonify({'response': response})
    return jsonify({'error': 'No message provided'}), 400

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False) # Important for Colab and subprocess
"""
    with open("app.py", "w") as f:
        f.write(app_py_content)

    # --- 3. Create requirements.txt ---
    requirements_content = """
Flask==2.3.3
Werkzeug==2.3.7
opencv-python==4.9.0.80 # Add if not already present for cv2
tensorflow==2.16.1 # Adjust version based on your ML model
onnxruntime==1.18.0 # For the chatbot
transformers==4.42.3 # For DistilBertTokenizerFast
scikit-learn==1.5.0 # For LabelEncoder
torch==2.3.1 # For torch.nn.functional.softmax"""

    with open("requirements.txt", "w") as f:
        f.write(requirements_content)

    # --- 4. Create templates/index.html ---
    index_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>DigitDetect AI: Intelligent Handwriting Recognition</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
    <link href="https://fonts.googleapis.com/css2?family=Space+Grotesk:wght@400;500;700&family=Orbitron:wght@700;900&display=swap" rel="stylesheet">
    <link href="https://unpkg.com/aos@2.3.1/dist/aos.css" rel="stylesheet">
    <link rel="icon" href="/static/images/favicon.ico" type="image/ico">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.3.0/font/bootstrap-icons.css">
</head>
<body>
    <header class="main-header">
        <div class="container">
            <h1 class="logo">DigitDetect AI</h1>
            <nav class="main-nav">
                <ul>
                    <li><a href="#hero" class="nav-link" data-text="Home">Home</a></li>
                    <li><a href="#upload-section" class="nav-link" data-text="Predict">Predict</a></li>
                    <li><a href="#features" class="nav-link" data-text="Features">Features</a></li>
                    <li><a href="#contact" class="nav-link" data-text="Contact">Contact</a></li>
                </ul>
            </nav>
            <button class="hamburger" aria-label="Menu"><span></span><span></span><span></span></button>
        </div>
    </header>

    <main>
        <section id="hero" class="hero-section">
            <div class="hero-backdrop"></div>
            <div class="container hero-content">
                <p class="hero-tagline">Unlocking the Future of Digits</p>
                <h2 class="hero-title">Intelligent Handwritten Digit Recognition</h2>
                <p class="hero-description">Experience the power of AI to instantly transform handwritten numbers into digital data. Fast, accurate, and incredibly intuitive.</p>
                <div class="hero-actions">
                    <a href="#upload-section" class="btn btn-primary animate-shine">Get Started</a>
                    <a href="#features" class="btn btn-secondary animate-border-pulse">Learn More</a>
                </div>
            </div>
        </section>

        <section id="upload-section" class="upload-section">
            <div class="container">
                <h3 class="section-title neon-green">Upload Your Digit & Predict</h3>
                <div class="upload-box">
                    <form id="prediction-form" action="/upload" method="post" enctype="multipart/form-data">
                        <div class="file-input-wrapper">
                            <input type="file" name="file" id="file-input" accept="image/*" required>
                            <label for="file-input" class="custom-file-upload">
                                <span class="icon">🖼️</span> Drag & Drop or Click to Upload
                            </label>
                            <span id="file-name" class="selected-file-name">No file selected. PNG, JPG, JFIF only.</span>
                        </div>
                        <div id="image-preview" class="image-preview-area">
                            <p>Image Preview</p>
                        </div>
                        <button type="submit" class="btn btn-predict animate-liquid" id="predict-button">Predict Digit</button>
                        <div class="loading-spinner" id="loading-spinner"></div>
                    </form>
                </div>
                {% with messages = get_flashed_messages(with_categories=true) %}
                    {% if messages %}
                        <ul class="flash-messages">
                            {% for category, message in messages %}
                                <li class="{{ category }}">
                                    {{ message }}
                                </li>
                            {% endfor %}
                        </ul>
                    {% endif %}
                {% endwith %}
            </div>
        </section>

        <section id="features" class="features-section">
            <div class="container">
                <h3 class="section-title neon-blue">Why Choose DigitDetect AI?</h3>
                <div class="features-grid">
                    <div class="feature-card">
                        <div class="feature-icon">⚡</div>
                        <h4>Lightning Fast Predictions</h4>
                        <p>Our optimized algorithms provide instant results, ensuring a seamless user experience.</p>
                    </div>
                    <div class="feature-card">
                        <div class="feature-icon">🎯</div>
                        <h4>Unrivaled Accuracy</h4>
                        <p>Built on robust deep learning models, capable of recognizing diverse handwriting styles with precision.</p>
                    </div>
                    <div class="feature-card">
                        <div class="feature-icon">🌐</div>
                        <h4>Continuous Service</h4>
                        <p>Available for use 24x7. No limits,no restrictions.</p>
                    </div>
                    <div class="feature-card">
                        <div class="feature-icon">🔒</div>
                        <h4>Privacy-Centric</h4>
                        <p>Your data is processed securely and is never stored, ensuring complete privacy.</p>
                    </div>
                    <div class="feature-card">
                        <div class="feature-icon">💰</div>
                        <h4>Free-to-Use</h4>
                        <p>Free Usage. Small support from DigitDetect to individuals ready to change the world.</p>
                    </div>
                        <div class="feature-card">
                        <div class="feature-icon">💻📱</div>
                        <h4>Intuitive Interface</h4>
                        <p>User Friendly Interface for enhancing their experience with DigitDetect.</p>
                    </div>
                </div>
            </div>
        </section>

        <section id="how-it-works" class="how-it-works-section">
            <div class="container">
                <h3 class="section-title neon-pink">How It Works</h3>
                <div class="steps-grid">
                    <div class="step-card">
                        <div class="step-number">1</div>
                        <h4>Upload Image</h4>
                        <p>Select your handwritten digit image using the intuitive upload interface.</p>
                    </div>
                    <div class="step-card">
                        <div class="step-number">2</div>
                        <h4>AI Processes</h4>
                        <p>Our deep learning model analyzes the image, extracting key features.</p>
                    </div>
                    <div class="step-card">
                        <div class="step-number">3</div>
                        <h4>Get Prediction</h4>
                        <p>Receive an instant prediction of the digit.</p>
                    </div>
                </div>
            </div>
        </section>

        <section id="contact" class="contact-section">
            <div class="container">
                <h3 class="section-title neon-orange">Have Questions? Get in Touch!</h3>
                <p class="contact-text">We're here to help you understand and integrate DigitDetect AI. Reach out to us!</p>
                <a href="mailto:info@DigitDetect.ai" class="btn btn-contact animate-draw-border">Email Us Now</a>
            </div>
        </section>
    </main>


    <footer class="main-footer">
        <div class="container">
            <p>&copy; 2025 DigitDetect AI. All rights reserved.</p>
            <div class="social-links">
                <a href="#"><i class="bi-facebook"></i></a>
                <a href="#"><i class="bi-twitter"></i></a>
                <a href="#"><i class="bi-linkedin"></i></a>
                <a href="#"><i class="bi-github"></i></a>
            </div>
        </div>
    </footer>

  <div id="chatbot-icon-container">
        <img src="{{ url_for('static', filename='images/bot.gif') }}" alt="Chatbot Icon">
    </div>

    <div id="chatbot-popup">
        <div class="chat-header">
            <p>DigitDetect AI Chatbot</p>
            <button id="chatbot-close-btn">&times;</button>
        </div>
        <div class="chat-messages" id="chat-messages">
            <div class="message bot-message">
                Hello! How can I help you today?
            </div>
        </div>
        <div class="chat-input-area">
            <input type="text" id="user-input" placeholder="Type your message..." autocomplete="off">
            <button id="send-button" class="btn-send">Send</button>
        </div>
    </div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/gsap/3.9.1/gsap.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/gsap/3.9.1/ScrollTrigger.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/gsap/3.9.1/MotionPathPlugin.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/gsap/3.9.1/ScrollToPlugin.min.js"></script>
    <script src="{{ url_for('static', filename='js/script.js') }}"></script>
</body>
</html>
"""
    with open("templates/index.html", "w") as f:
        f.write(index_html_content)

    # --- 5. Create templates/result.html ---
    result_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Prediction Result - DigitDetect AI</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
    <link href="https://fonts.googleapis.com/css2?family=Space+Grotesk:wght@400;500;700&family=Orbitron:wght@700;900&display=swap" rel="stylesheet">
    <link rel="icon" href="/static/images/favicon.ico" type="image/ico">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.3.0/font/bootstrap-icons.css">
</head>
<body>

    <header class="main-header">
        <div class="container">
            <h1 class="logo">DigitDetect AI</h1>
            <nav class="main-nav">
                <ul>
                    <li><a href="{{ url_for('index') }}#hero" class="nav-link" data-text="Home">Home</a></li>
                    <li><a href="{{ url_for('index') }}#upload-section" class="nav-link" data-text="Predict">Predict</a></li>
                    <li><a href="{{ url_for('index') }}#features" class="nav-link" data-text="Features">Features</a></li>
                    <li><a href="{{ url_for('index') }}#contact" class="nav-link" data-text="Contact">Contact</a></li>

                </ul>
            </nav>
       <button class="hamburger" aria-label="Menu"><span></span><span></span><span></span></button>
        </div>
    </header>

    <main>
        <section class="result-display-section">
            <div class="result-backdrop"></div>
            <div class="container result-content">
                <h3 class="section-title neon-purple">Prediction Complete!</h3>
                <p class="result-message">The AI has recognized your digit:</p>
                {% if prediction_file %}
                    <div class="processed-image-container">
                        <img src="{{ url_for('uploaded_file', filename=prediction_file.split('/')[-1]) }}" alt="Processed Image with Predictions" class="processed-image">
                    </div>
                {% endif %}
                <div class="result-actions">
                    <a href="{{ url_for('download_file', filename=prediction_file) }}" class="btn btn-primary animate-shine">Download Result</a>
                    <a href="/" class="btn btn-secondary animate-border-pulse">Try Another Image</a>
                </div>
            </div>
             {% with messages = get_flashed_messages(with_categories=true) %}
                {% if messages %}
                    <ul class="flash-messages result-flash">
                        {% for category, message in messages %}
                            <li class="{{ category }}">
                                {{ message }}
                            </li>
                        {% endfor %}
                    </ul>
                {% endif %}
            {% endwith %}
        </section>
    </main>

    <footer class="main-footer">
        <div class="container">
            <p>&copy; 2025 DigitDetect AI. All rights reserved.</p>
            <div class="social-links">
                <a href="#"><i class="bi-facebook"></i></a>
                <a href="#"><i class="bi-twitter"></i></a>
                <a href="#"><i class="bi-linkedin"></i></a>
                <a href="#"><i class="bi-github"></i></a>
            </div>
        </div>
    </footer>

    <div id="chatbot-icon-container">
        <img src="{{ url_for('static', filename='images/bot.gif') }}" alt="Chatbot Icon">
    </div>

    <div id="chatbot-popup">
        <div class="chat-header">
            <p>DigitDetect AI Chatbot</p>
            <button id="chatbot-close-btn">&times;</button>
        </div>
        <div class="chat-messages" id="chat-messages">
            <div class="message bot-message">
                Hello! How can I help you today?
            </div>
        </div>
        <div class="chat-input-area">
            <input type="text" id="user-input" placeholder="Type your message..." autocomplete="off">
            <button id="send-button" class="btn-send">Send</button>
        </div>
    </div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/gsap/3.9.1/gsap.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/gsap/3.9.1/ScrollTrigger.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/gsap/3.9.1/ScrollToPlugin.min.js"></script>
    <script src="{{ url_for('static', filename='js/script.js') }}"></script>
</body>
</html>
"""
    with open("templates/result.html", "w") as f:
        f.write(result_html_content)

    #--- Create chatbot.py---
    chatbot_code="""import json
import random
import numpy as np
import onnxruntime as ort
from transformers import DistilBertTokenizerFast
from sklearn.preprocessing import LabelEncoder
import torch # We need this for torch.nn.functional.softmax

# --- Configuration ---
TOKENIZER_PATH = "./fine_tuned_chatbot_model/" # Path to saved tokenizer files
LABEL_ENCODER_PATH = "./fine_tuned_chatbot_model/label_encoder.json"
QUANTIZED_ONNX_MODEL_PATH = "./quantized_chatbot_model.onnx"
RESPONSES_DATA_PATH = "./new_json.json" # Path to your original JSON for responses
MAX_SEQ_LENGTH = 128 # Must be the same as used during training and ONNX export!

# --- NEW: Fallback Threshold ---
# This is a critical value you'll need to tune.
# Start with 0.7 (70%) and adjust based on performance.
SOFTMAX_THRESHOLD = 0.7

# --- Load Assets ---
print("Loading assets...")
tokenizer = DistilBertTokenizerFast.from_pretrained(TOKENIZER_PATH)

# Load the label encoder
with open(LABEL_ENCODER_PATH, 'r') as f:
    loaded_classes = json.load(f)
label_encoder = LabelEncoder()
label_encoder.fit(loaded_classes) # Refit with the original classes

# Load the ONNX session
session = ort.InferenceSession(QUANTIZED_ONNX_MODEL_PATH, providers=['CPUExecutionProvider'])

# Load tag to responses mapping
tag_to_responses = {}
with open(RESPONSES_DATA_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)
for item in data:
    tag_to_responses[item['tag']] = item['output_text']

# Define fallback responses
fallback_responses = [
    "I'm sorry, I don't quite understand that. Could you please rephrase?",
    "I'm not sure what you mean. Can you provide more details or ask in a different way?",
    "My apologies, I couldn't process that request. Please try asking something else related to DigitDetect.",
    "I'm still learning! Can you simplify your question?",
    "That's a bit beyond my current capabilities. Is there something else I can help with?"
]

print("Assets loaded. Chatbot ready.")

# --- Inference Function ---
def chatbot_response(user_input):
    # Tokenize user input consistently with training and ONNX export
    inputs = tokenizer(
        user_input,
        return_tensors="np", # ONNX Runtime expects NumPy arrays
        padding='max_length',
        truncation=True,
        max_length=MAX_SEQ_LENGTH
    )

    # Get ONNX model inputs (input_ids and attention_mask)
    ort_inputs = {
        session.get_inputs()[0].name: inputs['input_ids'],
        session.get_inputs()[1].name: inputs['attention_mask']
    }

    # Run inference
    ort_outputs = session.run(None, ort_inputs)
    logits = ort_outputs[0] # Get the logits (first output), it's a numpy array

    # --- NEW: Convert logits to probabilities and check confidence ---
    # Convert numpy array to torch tensor for softmax
    logits_tensor = torch.tensor(logits)
    probabilities = torch.nn.functional.softmax(logits_tensor, dim=-1)

    # Get the highest probability and its index
    max_prob = torch.max(probabilities).item()
    predicted_class_id = torch.argmax(probabilities, dim=-1).item() # Use argmax on probabilities

    # --- Fallback Logic ---
    if max_prob < SOFTMAX_THRESHOLD:
        predicted_tag = "FALLBACK" # Assign a special tag for logging
        response = random.choice(fallback_responses)
    else:
        # Decode the predicted label
        predicted_tag = label_encoder.inverse_transform([predicted_class_id])[0]
        # Get a random response for the predicted tag
        responses = tag_to_responses.get(predicted_tag, ["I'm sorry, I didn't get that. Can you rephrase?"]) # Fallback if tag somehow missing
        response = random.choice(responses)

    return response
    """
    with open("chatbot.py","w") as f:
      f.write(chatbot_code)

    # --- 6. Create static/css/style.css ---
    style_css_content = """
/* --- Google Fonts --- */
@import url('https://fonts.googleapis.com/css2?family=Space+Grotesk:wght@400;500;700&family=Orbitron:wght@700;900&display=swap');

/* --- Variables --- */
:root {
    --primary-color: #6a5acd; /* Royal Purple */
    --secondary-color: #663399; /* Deep Amethyst */
    --dark-bg: #1a1a2e; /* Deep Dark Blue */
    --light-text: #e0e0e0;
    --dark-text: #c0c0c0;
    --card-bg: #22223b; /* Darker Purple */
    --border-radius-xl: 20px;
    --border-radius-lg: 15px;
    --border-radius-md: 10px;

    /* Neon Colors */
    --neon-pink: #ff69b4; /* Hot Pink */
    --neon-green: #39ff14; /* Electric Green */
    --neon-blue: #00ffff; /* Aqua Blue */
    --neon-orange: #ffa500; /* Bright Orange */
    --neon-purple: #bf00ff; /* Bright Purple */
}

/* Base Neon Glow Effect (can be overridden) */
.neon-glow {
    text-shadow: 0 0 5px var(--neon-pink), 0 0 10px var(--neon-pink), 0 0 15px var(--neon-pink), 0 0 20px var(--neon-pink);
}

/* Specific Neon Glows */
.neon-pink-glow { text-shadow: 0 0 5px var(--neon-pink), 0 0 10px var(--neon-pink), 0 0 15px var(--neon-pink); }
.neon-green-glow { text-shadow: 0 0 5px var(--neon-green), 0 0 10px var(--neon-green), 0 0 15px var(--neon-green); }
.neon-blue-glow { text-shadow: 0 0 5px var(--neon-blue), 0 0 10px var(--neon-blue), 0 0 15px var(--neon-blue); }
.neon-orange-glow { text-shadow: 0 0 5px var(--neon-orange), 0 0 10px var(--neon-orange), 0 0 15px var(--neon-orange); }
.neon-purple-glow { text-shadow: 0 0 5px var(--neon-purple), 0 0 10px var(--neon-purple), 0 0 15px var(--neon-purple); }


/* --- Base Styles --- */
body {
    font-family: 'Space Grotesk', sans-serif;
    margin: 0;
    padding: 0;
    background-color: var(--dark-bg);
    color: var(--light-text);
    line-height: 1.8;
    overflow-x: hidden;
    -webkit-font-smoothing: antialiased;
    -moz-osx-font-smoothing: grayscale;
}

.container {
    max-width: 1280px;
    margin: 0 auto;
    padding: 0 25px;
}

h1, h2, h3, h4 {
    font-family: 'Orbitron', sans-serif;
    color: var(--light-text);
}

h2 { font-size: 3.8em; margin-bottom: 0.6em; font-weight: 900; }
h3 { font-size: 2.8em; margin-bottom: 0.8em; font-weight: 700; }
p { margin-bottom: 1.2em; color: var(--dark-text); }

a {
    text-decoration: none;
    color: var(--primary-color);
    transition: color 0.3s ease;
}
a:hover {
    color: var(--neon-pink); /* Default link hover */
}

/* --- Header --- */
.main-header {
    background-color: rgba(26, 26, 46, 0.8); /* Semi-transparent dark bg */
    backdrop-filter: blur(10px);
    padding: 18px 0;
    position: fixed;
    width: 100%;
    top: 0;
    left: 0;
    z-index: 1000;
    box-shadow: 0 5px 20px rgba(0,0,0,0.4);
    transition: background-color 0.3s ease, box-shadow 0.3s ease;
}

.main-header .container {
    display: flex;
    justify-content: space-between;
    align-items: center;
}

.logo {
    font-size: 2.2em;
    margin: 0;
    letter-spacing: 2px;
    color: var(--primary-color);
    text-shadow: 0 0 8px var(--neon-pink);
    transition: transform 0.3s ease;
}
.logo:hover {
    transform: scale(1.05);
    text-shadow: var(--neon-pink-glow);
}

.main-nav ul {
    list-style: none;
    margin: 0;
    padding: 0;
    display: flex;
}

.main-nav ul li {
    margin-left: 45px;
}

.nav-link {
    color: var(--dark-text);
    font-weight: 500;
    font-size: 1.1em;
    position: relative;
    padding-bottom: 8px;
    transition: color 0.3s ease, transform 0.3s ease;
    overflow: hidden; /* For underline animation */
}

.nav-link::before {
    content: attr(data-text); /* Use data-text for hover effect */
    position: absolute;
    top: 0;
    left: 0;
    color: var(--neon-pink);
    transform: translateY(100%);
    transition: transform 0.3s ease;
}

.nav-link::after {
    content: '';
    position: absolute;
    bottom: 0;
    left: 0;
    width: 100%;
    height: 3px;
    background-color: var(--primary-color);
    transform: scaleX(0); /* Hidden by default */
    transform-origin: left;
    transition: transform 0.3s ease-out;
}

.nav-link:hover {
    color: transparent; /* Hide original text on hover */
    transform: translateY(-2px);
}
.nav-link:hover::before {
    transform: translateY(0);
}
.nav-link:hover::after {
    transform: scaleX(1); /* Show underline on hover */
}

.hamburger {
    display: none; /* Hidden on desktop */
    background: none;
    border: none;
    cursor: pointer;
    padding: 10px;
    position: relative;
    z-index: 1001;
}

.hamburger span {
    display: block;
    width: 30px;
    height: 3px;
    background-color: var(--primary-color);
    margin: 6px 0;
    transition: all 0.3s ease;
}

.hamburger.active span:nth-child(1) { transform: rotate(45deg) translate(5px, 5px); }
.hamburger.active span:nth-child(2) { opacity: 0; }
.hamburger.active span:nth-child(3) { transform: rotate(-45deg) translate(5px, -5px); }

/* Mobile Nav */
.main-nav.active {
    display: flex;
    flex-direction: column;
    position: absolute;
    top: 0;
    right: 0;
    width: 70%;
    height: 100vh;
    background-color: rgba(26, 26, 46, 0.95);
    backdrop-filter: blur(15px);
    padding-top: 100px;
    box-shadow: -10px 0 30px rgba(0,0,0,0.5);
    transform: translateX(0);
    animation: slideInRight 0.5s ease-out forwards;
}
.main-nav.active ul {
    flex-direction: column;
    align-items: center;
}
.main-nav.active ul li {
    margin: 20px 0;
}

@keyframes slideInRight {
    from { transform: translateX(100%); }
    to { transform: translateX(0); }
}


/* --- Hero Section --- */
.hero-section {
    position: relative;
    height: 100vh;
    display: flex;
    align-items: center;
    justify-content: center;
    text-align: center;
    overflow: hidden;
    padding-top: 100px;
    background: linear-gradient(145deg, #1a1a2e 0%, #0f0f1d 100%);
}

.hero-backdrop {
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: radial-gradient(circle at top left, rgba(106, 90, 205, 0.1) 0%, transparent 50%),
                radial-gradient(circle at bottom right, rgba(255, 105, 180, 0.1) 0%, transparent 50%);
    pointer-events: none;
}

.hero-content {
    position: relative;
    z-index: 2;
    max-width: 900px;
}

.hero-tagline {
    font-size: 1.4em;
    color: var(--neon-pink);
    letter-spacing: 2px;
    margin-bottom: 15px;
    font-weight: 500;
    text-shadow: 0 0 8px var(--neon-pink);
}

.hero-title {
    font-size: 4.8em;
    margin-bottom: 30px;
    line-height: 1.1;
    text-shadow: 0 0 20px var(--primary-color), 0 0 40px var(--neon-pink);
}

.hero-description {
    font-size: 1.3em;
    color: var(--dark-text);
    margin-bottom: 60px;
    max-width: 700px;
    margin-left: auto;
    margin-right: auto;
}

.hero-actions {
    display: flex;
    justify-content: center;
    gap: 30px;
}
/*
.shape {
    position: absolute;
    border-radius: 50%;
    filter: blur(50px);
}
.shape-1 { background-color: rgba(106, 90, 205, 0.1); }
.shape-2 { background-color: rgba(255, 105, 180, 0.1); }
.shape-3 { background-color: rgba(102, 51, 153, 0.1); }
*/

/* --- Buttons --- */
.btn {
    display: inline-block;
    padding: 18px 40px;
    border-radius: var(--border-radius-md);
    font-weight: 700;
    font-size: 1.15em;
    cursor: pointer;
    border: none;
    transition: all 0.4s cubic-bezier(0.25, 0.8, 0.25, 1);
    position: relative;
    overflow: hidden;
    letter-spacing: 0.5px;
    box-shadow: 0 5px 20px rgba(0,0,0,0.3);
}

.btn-primary {
    background: linear-gradient(45deg, var(--primary-color), var(--secondary-color));
    color: white;
}
.btn-secondary {
    background-color: transparent;
    border: 2px solid var(--primary-color);
    color: var(--primary-color);
}
.btn-predict {
    background: linear-gradient(90deg, var(--neon-pink), var(--primary-color));
    color: white;
    font-weight: 700;
    text-transform: uppercase;
}
.btn-contact {
    background-color: var(--neon-orange);
    color: var(--dark-bg);
    font-weight: 700;
}

/* Button Hover Effects & Animations */
.animate-shine::before {
    content: '';
    position: absolute;
    top: 0;
    left: -75%;
    width: 50%;
    height: 100%;
    background: rgba(255,255,255,0.3);
    transform: skewX(-20deg);
    transition: all 0.7s ease-in-out;
}
.animate-shine:hover::before {
    left: 125%;
}

.animate-border-pulse {
    animation: borderPulse 2s infinite ease-in-out;
}
@keyframes borderPulse {
    0% { border-color: var(--primary-color); box-shadow: 0 0 0 rgba(106,90,205,0.7); }
    50% { border-color: var(--neon-pink); box-shadow: 0 0 15px rgba(255,105,180,0.7); }
    100% { border-color: var(--primary-color); box-shadow: 0 0 0 rgba(106,90,205,0); }
}

.animate-liquid {
    position: relative;
    z-index: 1;
}
.animate-liquid::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: var(--secondary-color);
    z-index: -1;
    transform: scaleY(0);
    transform-origin: bottom;
    transition: transform 0.4s ease-out;
}
.animate-liquid:hover::before {
    transform: scaleY(1);
}
.animate-liquid:hover {
    color: white;
    transform: translateY(-3px);
}

.animate-draw-border {
    border: 2px solid transparent;
    background-clip: padding-box;
    transition: background-position 0.5s ease-out;
    background-image: linear-gradient(to right, var(--neon-orange) 0%, var(--primary-color) 100%);
    background-size: 200% 100%;
    background-position: right bottom;
}
.animate-draw-border:hover {
    background-position: left bottom;
    color: white;
}


/* --- Section General --- */
section {
    padding: 120px 0;
    text-align: center;
    position: relative;
    overflow: hidden;
    background-color: var(--dark-bg);
}

section:nth-child(odd) {
    background-color: #111122; /* Slightly different dark shade */
}

.section-title {
    margin-bottom: 60px;
    position: relative;
    display: inline-block;
    padding-bottom: 20px;
    font-weight: 700;
    font-size: 3.2em;
    color: var(--light-text);
}

/* Neon Titles */
.section-title.neon-green { color: var(--neon-green); text-shadow: var(--neon-green-glow); }
.section-title.neon-green::after { background: linear-gradient(90deg, var(--neon-green), var(--primary-color)); }

.section-title.neon-blue { color: var(--neon-blue); text-shadow: var(--neon-blue-glow); }
.section-title.neon-blue::after { background: linear-gradient(90deg, var(--neon-blue), var(--primary-color)); }

.section-title.neon-pink { color: var(--neon-pink); text-shadow: var(--neon-pink-glow); }
.section-title.neon-pink::after { background: linear-gradient(90deg, var(--neon-pink), var(--primary-color)); }

.section-title.neon-orange { color: var(--neon-orange); text-shadow: var(--neon-orange-glow); }
.section-title.neon-orange::after { background: linear-gradient(90deg, var(--neon-orange), var(--primary-color)); }

.section-title.neon-purple { color: var(--neon-purple); text-shadow: var(--neon-purple-glow); }
.section-title.neon-purple::after { background: linear-gradient(90deg, var(--neon-purple), var(--primary-color)); }


.section-title::after {
    content: '';
    position: absolute;
    left: 50%;
    transform: translateX(-50%);
    bottom: 0;
    width: 100px;
    height: 5px;
    background: linear-gradient(90deg, var(--neon-pink), var(--primary-color)); /* Default */
    border-radius: 3px;
}

/* --- Upload Section --- */
.upload-section {
    background: linear-gradient(160deg, #111122 0%, #0a0a1a 100%);
    padding: 80px 0;
}

.upload-box {
    background-color: var(--card-bg);
    border-radius: var(--border-radius-xl);
    padding: 60px;
    max-width: 900px;
    margin: 50px auto 0;
    box-shadow: 0 10px 40px rgba(0,0,0,0.5);
    border: 1px solid rgba(255,255,255,0.05);
    position: relative;
    z-index: 1;
    overflow: hidden;
}
.upload-box::before {
    content: '';
    position: absolute;
    top: -50%;
    left: -50%;
    width: 200%;
    height: 200%;
    background: radial-gradient(circle at center, rgba(106, 90, 205, 0.1) 0%, transparent 60%);
    animation: rotateGradient 20s linear infinite;
    z-index: -1;
}

@keyframes rotateGradient {
    from { transform: rotate(0deg); }
    to { transform: rotate(360deg); }
}


#prediction-form {
    display: flex;
    flex-direction: column;
    align-items: center;
    gap: 35px;
}

.file-input-wrapper {
    position: relative;
    width: 90%;
    max-width: 600px;
    height: 150px;
    border: 3px dashed var(--primary-color);
    border-radius: var(--border-radius-lg);
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    cursor: pointer;
    overflow: hidden;
    transition: all 0.4s cubic-bezier(0.25, 0.8, 0.25, 1);
    background-color: rgba(255,255,255,0.05);
    box-shadow: inset 0 2px 10px rgba(0,0,0,0.3);
}

.file-input-wrapper:hover {
    border-color: var(--neon-green); /* Neon Green Hover */
    background-color: rgba(57,255,20,0.08);
    transform: translateY(-5px);
    box-shadow: inset 0 2px 15px rgba(0,0,0,0.4), 0 5px 20px rgba(0,0,0,0.4);
}

#file-input {
    opacity: 0;
    position: absolute;
    width: 100%;
    height: 100%;
    cursor: pointer;
}

.custom-file-upload {
    display: flex;
    flex-direction: column;
    align-items: center;
    font-size: 1.4em;
    color: var(--light-text);
    font-weight: 600;
    cursor: pointer;
    transition: color 0.3s ease;
}

.file-input-wrapper:hover .custom-file-upload {
    color: var(--neon-green);
}

.custom-file-upload .icon {
    font-size: 3.5em;
    margin-bottom: 10px;
    color: var(--primary-color);
    transition: transform 0.4s ease, color 0.3s ease;
}
.file-input-wrapper:hover .custom-file-upload .icon {
    transform: scale(1.1) rotate(5deg);
    color: var(--neon-green);
}

.selected-file-name {
    margin-top: 20px;
    font-size: 1em;
    color: var(--dark-text);
    transition: color 0.3s ease;
}

.image-preview-area {
    width: 250px;
    height: 250px;
    border: 2px solid rgba(255,255,255,0.1);
    background-color: rgba(0,0,0,0.1);
    display: flex;
    justify-content: center;
    align-items: center;
    overflow: hidden;
    border-radius: var(--border-radius-md);
    margin-top: 30px;
    box-shadow: inset 0 2px 8px rgba(0,0,0,0.5);
    position: relative;
    transform: scale(0.9); /* Initial state for animation */
    opacity: 0;
    transition: transform 0.5s ease-out, opacity 0.5s ease-out;
}

.image-preview-area.has-image {
    transform: scale(1);
    opacity: 1;
}

.image-preview-area img {
    max-width: 100%;
    max-height: 100%;
    object-fit: contain;
    border-radius: var(--border-radius-md);
    filter: grayscale(100%); /* Start grayscale */
    transition: filter 0.5s ease-out;
}
.image-preview-area.has-image img {
    filter: grayscale(0%); /* Remove grayscale on load */
}

.loading-spinner {
    border: 6px solid rgba(255, 255, 255, 0.1);
    border-top-color: var(--neon-pink); /* Loading spinner uses pink */
    border-radius: 50%;
    width: 50px;
    height: 50px;
    animation: spin 0.9s linear infinite;
    display: none;
    margin-top: 30px;
    box-shadow: var(--neon-pink-glow);
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

/* --- Features Section --- */
.features-section {
    background: linear-gradient(135deg, #111122 0%, #0a0a1a 100%);
}

.features-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
    gap: 40px;
    margin-top: 60px;
}

.feature-card {
    background-color: var(--card-bg);
    padding: 40px;
    border-radius: var(--border-radius-lg);
    box-shadow: 0 5px 25px rgba(0,0,0,0.4);
    transition: transform 0.4s cubic-bezier(0.25, 0.8, 0.25, 1), box-shadow 0.4s ease;
    border: 1px solid rgba(255,255,255,0.05);
    display: flex;
    flex-direction: column;
    align-items: center;
    text-align: center;
    position: relative;
    overflow: hidden;
}
.feature-card::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: linear-gradient(45deg, var(--primary-color), var(--neon-blue)); /* Feature cards use Neon Blue */
    opacity: 0;
    transition: opacity 0.3s ease;
    z-index: 0;
}
.feature-card:hover::before {
    opacity: 0.1;
}
.feature-card:hover {
    transform: translateY(-15px) scale(1.02);
    box-shadow: 0 15px 40px rgba(0,0,0,0.6), 0 0 15px var(--neon-blue), 0 0 30px var(--neon-blue);
}

.feature-icon {
    font-size: 4em;
    margin-bottom: 25px;
    color: var(--neon-blue); /* Feature icons use Neon Blue */
    position: relative;
    z-index: 1;
    text-shadow: var(--neon-blue-glow);
}

.feature-card h4 {
    font-size: 1.8em;
    margin-bottom: 15px;
    color: var(--light-text);
    position: relative;
    z-index: 1;
}

.feature-card p {
    font-size: 0.95em;
    color: var(--dark-text);
    position: relative;
    z-index: 1;
}

/* --- How It Works Section --- */
.how-it-works-section {
    background: linear-gradient(160deg, #0a0a1a 0%, #00000a 100%);
    padding-bottom: 100px;
}

.steps-grid {
    display: flex;
    justify-content: center;
    gap: 40px;
    margin-top: 60px;
    flex-wrap: wrap;
}

.step-card {
    background-color: var(--card-bg);
    border-radius: var(--border-radius-lg);
    padding: 35px;
    max-width: 320px;
    box-shadow: 0 5px 20px rgba(0,0,0,0.3);
    border: 1px solid rgba(255,255,255,0.05);
    position: relative;
    overflow: hidden;
    transition: transform 0.3s ease, box-shadow 0.3s ease;
}
.step-card:hover {
    transform: translateY(-10px);
    box-shadow: 0 15px 40px rgba(0,0,0,0.6), 0 0 15px var(--neon-pink), 0 0 30px var(--neon-pink);
}

.step-number {
    font-family: 'Orbitron', sans-serif;
    font-size: 3.5em;
    font-weight: 900;
    color: var(--neon-pink); /* Steps use Neon Pink */
    margin-bottom: 15px;
    text-shadow: var(--neon-pink-glow);
    line-height: 1;
    position: relative;
    z-index: 1;
}
.step-card h4 {
    font-size: 1.8em;
    color: var(--light-text);
    margin-bottom: 10px;
    position: relative;
    z-index: 1;
}
.step-card p {
    font-size: 0.95em;
    color: var(--dark-text);
    position: relative;
    z-index: 1;
}

/* --- Contact Section --- */
.contact-section {
    background: linear-gradient(145deg, #00000a 0%, #1a1a2e 100%);
    padding-bottom: 120px;
}

.contact-text {
    max-width: 800px;
    margin: 0 auto 50px auto;
    font-size: 1.2em;
    color: var(--dark-text);
}

/* --- Flash Messages --- */
.flash-messages {
    list-style: none;
    padding: 0;
    margin-top: 40px;
    max-width: 800px;
    margin-left: auto;
    margin-right: auto;
    text-align: left;
}

.flash-messages li {
    padding: 18px 25px;
    margin-bottom: 15px;
    border-radius: var(--border-radius-md);
    font-weight: 500;
    font-size: 1.05em;
    display: flex;
    align-items: center;
    box-shadow: 0 4px 15px rgba(0,0,0,0.2);
    animation: slideInFromTop 0.5s ease-out forwards;
}

@keyframes slideInFromTop {
    from { opacity: 0; transform: translateY(-20px); }
    to { opacity: 1; transform: translateY(0); }
}

.flash-messages li::before {
    margin-right: 15px;
    font-size: 1.5em;
}

.flash-messages li.success {
    background-color: #2e8b57; /* Sea Green */
    color: white;
    border: 1px solid #3cb371;
}
.flash-messages li.success::before { content: '✔️'; }

.flash-messages li.error {
    background-color: #dc143c; /* Crimson */
    color: white;
    border: 1px solid #ff6347;
}
.flash-messages li.error::before { content: '❌'; }

.result-flash { /* Specific styling for flash messages on result page */
    position: absolute;
    bottom: 30px;
    left: 50%;
    transform: translateX(-50%);
    width: 90%;
    max-width: 600px;
}

/* --- Result Page --- */
.result-display-section {
    min-height: 100vh;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    padding: 120px 0;
    background: linear-gradient(160deg, #0f0f1d 0%, #1a1a2e 100%);
    position: relative;
    overflow: hidden;
}

.result-backdrop {
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: radial-gradient(circle at center, rgba(106, 90, 205, 0.1) 0%, transparent 60%);
    animation: pulseBg 10s infinite alternate;
    pointer-events: none;
}
@keyframes pulseBg {
    0% { transform: scale(1); opacity: 0.5; }
    100% { transform: scale(1.05); opacity: 0.6; }
}

.result-content {
    background-color: var(--card-bg);
    padding: 70px;
    border-radius: var(--border-radius-xl);
    box-shadow: 0 15px 50px rgba(0,0,0,0.6);
    text-align: center;
    max-width: 700px;
    border: 2px solid var(--primary-color);
    position: relative;
    z-index: 1;
}

.result-message {
    font-size: 1.4em;
    color: var(--dark-text);
    margin-bottom: 25px;
}

.predicted-digit-display {
    font-family: 'Orbitron', sans-serif;
    font-size: 10em;
    font-weight: 900;
    color: var(--neon-purple); /* Result digit uses Neon Purple */
    margin: 30px 0 25px;
    text-shadow: var(--neon-purple-glow);
}

.result-actions {
    display: flex;
    justify-content: center;
    gap: 30px;
    margin-top: 40px;
}


/* --- Chatbot Pop-up & Icon --- */
#chatbot-icon-container {
    position: fixed;
    bottom: 25px;
    right: 25px;
    width: 60px;
    height: 60px;
    border-radius: 50%;
    cursor: pointer;
    overflow: hidden;
    box-shadow: 0 4px 12px rgba(0,0,0,0);
    z-index: 1000;
    transition: transform 0.3s ease;
}

#chatbot-icon-container img {
    width: 100%;
    height: 100%;
    object-fit: cover;
}

#chatbot-icon-container:hover {
    transform: scale(1.1);
}

#chatbot-popup {
    position: fixed;
    bottom: 95px; /* Positioned just above the icon */
    right: 25px;
    width: 350px;
    height: 450px;
    background-color: var(--card-bg);
    border-radius: var(--border-radius-xl);
    box-shadow: 0 10px 40px rgba(0,0,0,0.5);
    border: 1px solid rgba(255,255,255,0.05);
    z-index: 1000;

    /* Initially hidden */
    transform: translateY(100%) scale(0.5);
    opacity: 0;
    visibility: hidden;
    transition: all 0.4s cubic-bezier(0.68, -0.55, 0.27, 1.55);
    display: flex;
    flex-direction: column;
}

#chatbot-popup.show-chatbot {
    transform: translateY(0) scale(1);
    opacity: 1;
    visibility: visible;
}

.chat-header {
    background-color: var(--primary-color);
    color: white;
    padding: 15px;
    border-radius: var(--border-radius-md) var(--border-radius-md) 0 0;
    font-weight: 700;
    font-size: 1.2em;
    text-align: center;
    position: relative;
    box-shadow: 0 2px 10px rgba(0,0,0,0.3);
}

#chatbot-close-btn {
    position: absolute;
    top: 50%;
    right: 15px;
    transform: translateY(-50%);
    background: transparent;
    border: none;
    color: white;
    font-size: 1.5em;
    cursor: pointer;
    line-height: 1;
}

.chat-messages {
    flex-grow: 1;
    overflow-y: auto;
    padding: 15px;
    display: flex;
    flex-direction: column;
    gap: 15px;
}

.message {
    padding: 12px 18px;
    border-radius: var(--border-radius-md);
    max-width: 80%;
    word-wrap: break-word;
    font-size: 0.95em;
    line-height: 1.5;
    box-shadow: 0 2px 5px rgba(0,0,0,0.2);
}

.user-message {
    background-color: var(--neon-blue);
    color: var(--dark-bg);
    align-self: flex-end;
    border-bottom-right-radius: 5px;
}

.bot-message {
    background-color: #33334f;
    color: var(--light-text);
    align-self: flex-start;
    border-bottom-left-radius: 5px;
}

.chat-input-area {
    display: flex;
    padding: 15px;
    gap: 10px;
    border-top: 1px solid rgba(255,255,255,0.1);
}

.chat-input-area input[type="text"] {
    flex-grow: 1;
    padding: 12px 15px;
    border: 1px solid var(--primary-color);
    border-radius: var(--border-radius-md);
    background-color: rgba(255,255,255,0.08);
    color: var(--light-text);
    font-size: 1em;
    outline: none;
    transition: border-color 0.3s ease, background-color 0.3s ease;
}

.chat-input-area input[type="text"]:focus {
    border-color: var(--neon-green);
    background-color: rgba(57,255,20,0.05);
}

.btn-send {
    background: linear-gradient(45deg, var(--neon-green), var(--primary-color));
    color: white;
    border: none;
    padding: 12px 25px;
    border-radius: var(--border-radius-md);
    cursor: pointer;
    font-weight: 600;
    font-size: 1em;
    transition: all 0.3s ease;
}

.btn-send:hover {
    opacity: 0.9;
    transform: translateY(-2px);
    box-shadow: 0 5px 15px rgba(0,0,0,0.3);
}

.bot-message.loading {
    background-color: #4a4a6b;
    font-style: italic;
    color: var(--dark-text);
    animation: pulse 1.5s infinite ease-in-out;
}

@keyframes pulse {
    0% { opacity: 0.7; }
    50% { opacity: 1; }
    100% { opacity: 0.7; }
}

.chat-messages::-webkit-scrollbar {
    width: 8px;
}

.chat-messages::-webkit-scrollbar-track {
    background: var(--card-bg);
    border-radius: 10px;
}

.chat-messages::-webkit-scrollbar-thumb {
    background: var(--primary-color);
    border-radius: 10px;
}

.chat-messages::-webkit-scrollbar-thumb:hover {
    background: var(--neon-pink);
}

@media (max-width: 768px) {
    #chatbot-popup {
        width: 90%;
        height: 80vh;
        right: 5%;
        bottom: 80px;
        left: 5%;
    }
}

/* --- Footer --- */
.main-footer {
    background-color: #0d0d1a; /* Even darker shade */
    color: var(--dark-text);
    text-align: center;
    padding: 35px 0;
    font-size: 0.9em;
    border-top: 1px solid rgba(255,255,255,0.05);
}
.social-links {
    margin-top: 20px;
    display: flex;
    justify-content: center;
    gap: 15px;
}


/* --- Responsive Design --- */
@media (max-width: 1024px) {
    .hero-title { font-size: 3.5em; }
    .hero-description { font-size: 1.1em; }
    .main-nav ul li { margin-left: 30px; }
    .section-title { font-size: 2.5em; }
    .predicted-digit-display { font-size: 8em; }
}

@media (max-width: 768px) {
    .main-header .container {
        flex-direction: row; /* Keep logo and hamburger on one row */
        justify-content: space-between;
    }
    .main-nav { display: none; } /* Hide by default */
    .hamburger { display: block; } /* Show hamburger */

    .hero-section { text-align: left; padding: 100px 20px; }
    .hero-content { align-items: flex-start; text-align: left; }
    .hero-title { font-size: 2.8em; }
    .hero-description { font-size: 1em; text-align: left; margin-left: 0; margin-right: 0;}
    .hero-actions { justify-content: flex-start; flex-direction: column; gap: 20px;}
    .btn { padding: 15px 30px; font-size: 1em; width: 100%; max-width: 300px;} /* Full width buttons */

    .section-title { font-size: 2em; margin-bottom: 40px; }
    .upload-box { padding: 40px 20px; }
    .file-input-wrapper { height: 120px; font-size: 1.2em; }
    .custom-file-upload .icon { font-size: 3em; }
    .image-preview-area { width: 150px; height: 150px; }

    .features-grid, .steps-grid { grid-template-columns: 1fr; gap: 30px; }
    .feature-card, .step-card { max-width: 100%; }

    .predicted-digit-display { font-size: 6em; }
    .result-content { padding: 40px 25px; }
    .result-actions { flex-direction: column; gap: 20px; }
}

@media (max-width: 480px) {
    .logo { font-size: 1.8em; }
    .hero-title { font-size: 2.2em; }
    .hero-description { font-size: 0.9em; }
    .section-title { font-size: 1.8em; }
    .file-input-wrapper { height: 100px; font-size: 1em; }
    .custom-file-upload .icon { font-size: 2.5em; }
    .image-preview-area { width: 150px; height: 150px; }
    .predicted-digit-display { font-size: 5em; }
    .contact-text { font-size: 1em; }
    .social-icon { width: 40px; height: 40px; font-size: 1em; }
}
"""
    with open("static/css/style.css", "w") as f:
        f.write(style_css_content)

    # --- 7. Create static/js/script.js ---
    script_js_content = """
document.addEventListener('DOMContentLoaded', function() {
    const fileInput = document.getElementById('file-input');
    const fileNameSpan = document.getElementById('file-name');
    const imagePreview = document.getElementById('image-preview');
    const predictionForm = document.getElementById('prediction-form');
    const loadingSpinner = document.getElementById('loading-spinner');
    const predictButton = document.getElementById('predict-button');
    const hamburger = document.querySelector('.hamburger');
    const mainNav = document.querySelector('.main-nav');
    const navLinks = document.querySelectorAll('.nav-link');
    const header = document.querySelector('.main-header');

    // Register GSAP plugins
    gsap.registerPlugin(ScrollTrigger, ScrollToPlugin);

    // --- Hamburger Menu Toggle ---
    if (hamburger && mainNav) {
        hamburger.addEventListener('click', () => {
            hamburger.classList.toggle('active');
            mainNav.classList.toggle('active');
            // Prevent body scroll when menu is open
            document.body.style.overflow = mainNav.classList.contains('active') ? 'hidden' : '';
        });

        // Close menu when a nav link is clicked (for mobile)
        navLinks.forEach(link => {
            link.addEventListener('click', () => {
                hamburger.classList.remove('active');
                mainNav.classList.remove('active');
                document.body.style.overflow = '';
            });
        });
    }

    // --- File Input and Image Preview ---
    if (fileInput && fileNameSpan && imagePreview) {
        fileInput.addEventListener('change', function() {
            if (this.files && this.files.length > 0) {
                const file = this.files[0];
                fileNameSpan.textContent = file.name;
                imagePreview.classList.add('has-image'); // Add class for animation

                const reader = new FileReader();
                reader.onload = function(e) {
                    imagePreview.innerHTML = `<img src="${e.target.result}" alt="Image Preview">`;
                };
                reader.readAsDataURL(file);
            } else {
                fileNameSpan.textContent = 'No file selected. PNG, JPG, JFIF only.';
                imagePreview.innerHTML = '<p>Image Preview</p>'; // Reset preview
                imagePreview.classList.remove('has-image'); // Remove class
            }
        });
    }

    // --- Form Submission: Show Loading Spinner & Disable Button ---
    if (predictionForm && loadingSpinner && predictButton && fileInput) {
        predictionForm.addEventListener('submit', function(event) {
            if (fileInput.files.length === 0) {
                // Flash message for no file selected is handled by Flask
                return;
            }
            // Add a class to button for animation during submission
            predictButton.classList.add('is-loading');
            loadingSpinner.style.display = 'block';
            predictButton.disabled = true;
            predictButton.textContent = 'Predicting...';
            predictButton.style.cursor = 'not-allowed';

            // Clear previous flash messages immediately
            const flashMessages = document.querySelector('.flash-messages');
            if (flashMessages) {
                flashMessages.innerHTML = '';
            }
        });
    }

    // --- Smooth Scrolling for Navigation Links with GSAP ---
    navLinks.forEach(anchor => {
        anchor.addEventListener('click', function (e) {
             // Get the full href attribute
            const href = this.getAttribute('href');

             // Check if the link is an in-page anchor (starts with '#' and is on the current page)
            if (href.startsWith('#') && window.location.pathname === '/') {
                e.preventDefault(); // ONLY prevent the default if it's an in-page link on the homepage

            const targetId = this.getAttribute('href').substring(1);
            const targetElement = document.getElementById(targetId);

            if (targetElement) {
                const headerOffset = header.offsetHeight;
                const offsetPosition = targetElement.getBoundingClientRect().top + window.pageYOffset - headerOffset - 20;

                gsap.to(window, {
                    duration: 1.2,
                    scrollTo: {
                        y: offsetPosition,
                        autoKill: false
                        // ease: "power2.out" // Already defined in ScrollToPlugin defaults
                    }
                });
              }
            }
        });
    });

    // --- GSAP ScrollTrigger Animations with Reversibility ---

    // Sticky Header Shadow on Scroll
    gsap.to(header, {
        boxShadow: "0 8px 30px rgba(0,0,0,0.5)",
        backgroundColor: "rgba(26, 26, 46, 0.95)",
        duration: 0.3,
        scrollTrigger: {
            trigger: "body",
            start: "top -=50",
            end: "top -=51",
            toggleActions: "play reverse none reverse", // Play on enter, reverse on leave
        }
    });

    // Hero Section Animations
    const heroTl = gsap.timeline({
        scrollTrigger: {
            trigger: "#hero",
            start: "top 80%", // When top of hero is 80% down from viewport top
            end: "bottom top", // Ends when hero leaves viewport
            toggleActions: "play reverse play reverse", // Play on enter, reverse on leave
            scrub: 1, // Smoothly link animation to scroll position
            // markers: true // Uncomment for debugging
        }
    });

    heroTl.from(".hero-content .hero-tagline, .hero-content .hero-title, .hero-content .hero-description, .hero-content .hero-actions", {
        opacity: 0,
        y: 50,
        stagger: 0.2,
        ease: "power3.out"
    }, 0); // Stagger animations for content

/*    // Background Shapes Animation
    gsap.to(".shape-1", {
        y: 50, x: -50, scale: 1.2,
        repeat: -1, yoyo: true, ease: "power1.inOut", duration: 15
    });
    gsap.to(".shape-2", {
        y: -70, x: 70, scale: 1.1,
        repeat: -1, yoyo: true, ease: "power2.inOut", duration: 18
    });
    gsap.to(".shape-3", {
        y: 60, x: 60, scale: 1.3,
        repeat: -1, yoyo: true, ease: "power3.inOut", duration: 12
    });
*/

    // Upload Section Animations
    gsap.from(".upload-section .section-title", {
        opacity: 0,
        y: -50,
        ease: "power2.out",
        scrollTrigger: {
            trigger: ".upload-section",
            start: "top 80%",
            end: "top 40%",
            toggleActions: "play reverse play reverse",
            scrub: 1
        }
    });

    gsap.from(".upload-box", {
        opacity: 0,
        scale: 0.8,
        ease: "back.out(1.7)",
        scrollTrigger: {
            trigger: ".upload-box",
            start: "top 85%",
            end: "top 50%",
            toggleActions: "play reverse play reverse",
            scrub: 1
        }
    });

    gsap.from(".file-input-wrapper", {
        opacity: 0,
        y: 50,
        ease: "power2.out",
        scrollTrigger: {
            trigger: ".file-input-wrapper",
            start: "top 90%",
            toggleActions: "play reverse play reverse",
            scrub: 1
        }
    });


    // Features Section Animations
    gsap.from(".features-section .section-title", {
        opacity: 0,
        y: -50,
        ease: "power2.out",
        scrollTrigger: {
            trigger: ".features-section",
            start: "top 80%",
            end: "top 40%",
            toggleActions: "play reverse play reverse",
            scrub: 1
        }
    });

    gsap.from(".feature-card", {
        opacity: 0,
        y: 100,
        stagger: 0.15, // Stagger cards
        ease: "power2.out",
        scrollTrigger: {
            trigger: ".features-grid",
            start: "top 85%",
            end: "bottom 60%", // End animation when grid is mostly out of view
            toggleActions: "play reverse play reverse",
            scrub: 1
        }
    });

    // How It Works Section Animations
    gsap.from(".how-it-works-section .section-title", {
        opacity: 0,
        y: -50,
        ease: "power2.out",
        scrollTrigger: {
            trigger: ".how-it-works-section",
            start: "top 80%",
            end: "top 40%",
            toggleActions: "play reverse play reverse",
            scrub: 1
        }
    });

    gsap.from(".step-card", {
        opacity: 0,
        x: -100, // Animate from left
        stagger: 0.2,
        ease: "power2.out",
        scrollTrigger: {
            trigger: ".steps-grid",
            start: "top 85%",
            end: "bottom 60%",
            toggleActions: "play reverse play reverse",
            scrub: 1
        }
    });
       // --- Chatbot Specific JavaScript for the Pop-up ---
    const chatbotIcon = document.getElementById('chatbot-icon-container');
    const chatbotPopup = document.getElementById('chatbot-popup');
    const chatMessages = document.getElementById('chat-messages');
    const userInput = document.getElementById('user-input');
    const sendButton = document.getElementById('send-button');
    const chatbotCloseBtn = document.getElementById('chatbot-close-btn');

    // Toggle the pop-up window
    if (chatbotIcon && chatbotPopup) {
        chatbotIcon.addEventListener('click', () => {
            chatbotPopup.classList.toggle('show-chatbot');
        });

        chatbotCloseBtn.addEventListener('click', () => {
            chatbotPopup.classList.remove('show-chatbot');
        });
    }

    function addMessage(sender, message) {
        const messageDiv = document.createElement('div');
        messageDiv.classList.add('message');
        messageDiv.classList.add(sender === 'user' ? 'user-message' : 'bot-message');
        messageDiv.textContent = message;
        chatMessages.appendChild(messageDiv);
        chatMessages.scrollTop = chatMessages.scrollHeight;
    }

    async function sendMessage() {
        const message = userInput.value.trim();
        if (message === '') return;

        addMessage('user', message);
        userInput.value = '';

        const loadingMessageDiv = document.createElement('div');
        loadingMessageDiv.classList.add('message', 'bot-message', 'loading');
        loadingMessageDiv.innerHTML = 'Thinking...';
        chatMessages.appendChild(loadingMessageDiv);
        chatMessages.scrollTop = chatMessages.scrollHeight;

        try {
            const response = await fetch('/chat', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ message: message }),
            });

            const data = await response.json();

            chatMessages.removeChild(loadingMessageDiv);

            if (data.response) {
                addMessage('bot', data.response);
            } else if (data.error) {
                addMessage('bot', `Error: ${data.error}`);
            }
        } catch (error) {
            console.error('Error sending message:', error);
            chatMessages.removeChild(loadingMessageDiv);
            addMessage('bot', 'Oops! Something went wrong. Please try again.');
        }
    }

    if (sendButton) {
        sendButton.addEventListener('click', sendMessage);
    }

    if (userInput) {
        userInput.addEventListener('keypress', (e) => {
            if (e.key === 'Enter') {
                sendMessage();
            }
        });
    }



    // Contact Section Animations
    gsap.from(".contact-section .section-title, .contact-text, .btn-contact", {
        opacity: 0,
        y: 50,
        stagger: 0.2,
        ease: "power2.out",
        scrollTrigger: {
            trigger: ".contact-section",
            start: "top 80%",
            end: "top 50%",
            toggleActions: "play reverse play reverse",
            scrub: 1
        }
    });

    // Result Page specific animations (if applicable, loaded on result.html)
    // These need to be managed separately as they are on a different page.
    // However, the GSAP setup is there if you want to add more to result.html
    if (document.querySelector('.result-display-section')) {
        gsap.from(".result-content", {
            opacity: 0,
            scale: 0.8,
            y: 50,
            ease: "back.out(1.7)",
            duration: 1
        });
        gsap.from(".predicted-digit-display", {
            opacity: 0,
            scale: 0.5,
            ease: "elastic.out(1, 0.5)",
            delay: 0.5,
            duration: 1.5
        });
    }

    // Flash messages animation (already in CSS with keyframes, but can be GSAP too)
    gsap.from(".flash-messages li", {
        opacity: 0,
        y: -20,
        stagger: 0.1,
        duration: 0.5,
        ease: "power2.out"
    });

});
"""
    with open("static/js/script.js", "w") as f:
        f.write(script_js_content)

def install_python_dependencies():
    """Installs required Python packages from requirements.txt."""
    try:
        subprocess.run(["pip", "install", "-r", "requirements.txt", "-q", "--no-warn-script-location"],
                       check=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        raise SystemExit("Required Python packages could not be installed.")


def main():
    create_flask_app_files_v3()
    install_python_dependencies()

if __name__ == "__main__":
    main()

In [ ]:
!wget -q -O - ipv4.icanhazip.com
!python app.py & npx localtunnel --port 5000